In [1]:
from hyde_utils import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solut

In [2]:

dataset = load_dataset_hyde()
encoder = get_hyde_encoder()

dataset = process_dataset(dataset, encoder)




Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-0968d8a1246c8167/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0968d8a1246c8167/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-b6f9a9c9d8a7a8fe.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0968d8a1246c8167/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-08b937466dd23944.arrow


  0%|          | 0/1 [00:00<?, ?it/s]

120836
120836


In [23]:
from collections import defaultdict
applications = []
bad_count = 0
app_name_dict = defaultdict(list)
for i in range(len(dataset)):
    if "Applications of a" in dataset[i]['text']: 
        app_line = dataset[i]['text'].split("\n")[3].split("Applications of a")[1]
        try:
            name, app_list = app_line.split(":")
        except:
            print(name)
            print(app_line)
            bad_count += 1
            continue
        name = name.strip()
        app_list = [x.strip() for x in app_list.split(",")]
        for app in app_list:
            app_name_dict[app].append(name)

Consumables for Direct Fastening
 Intumescent Acoustic Mastic FiAM: Metallic pipes: 13" (325 mm),Cable trays: 18" x 2" (450 x 50 mm),Cable bunches: 4" (100 mm),Linear joints: flexible and rigid construction elements,Joints between FCPS coated panel system
Consumables for Direct Fastening
 Intumescent Acoustic Mastic FiAM US: Linear joints: flexible and rigid construction elements with dynamic movement,Metallic pipes and ducts,Insulated metallic pipes,Conduits,Cable and cable bunches,Cable trays
Fire Rated Silicone Sealant FFRS
 Rapid Fire Seal RFS 640: Curtain wall/slab edge: 8" (200 mm),Head of wall: 4" (100 mm),General construction joints: 8" (200 mm),Cable tray: 24" x 4" (600 mm x 100 mm),Steel pipes: 8" (200 mm)
Fire I Barr ElastoSeal FFB-ES
 Universal FireStopping Sealant UFS: Metallic services: steel and cast iron 20“ (500 mm) - copper 6“ (150 mm),Non metallic service: PVC 2“ (51 mm open) 3“ (75 mm closed),Insulated service: 20“ (500 mm),Construction joints: 4“ (100 mm),HVAC 100“

In [24]:
bad_count

9

In [28]:
len(app_name_dict)

830

In [35]:
keys = list(app_name_dict.keys())
for k in keys:
    if len(app_name_dict[k]) < 2:
        del app_name_dict[k]

In [36]:
len(app_name_dict)

291

In [66]:
def convert_dict2context(d):
    start_context = "Application of products: \n"
    for k in d:
        current_line = ""
        current_line += k 
        current_line += ":"+",".join(d[k][:4])
        start_context += current_line + "\n"
    return start_context

In [67]:
q = "Hi, I want to mount my TV on a wall, what would you recommend?"
filtered_dict = defaultdict(list)

In [68]:
from fuzzywuzzy import fuzz
scores = []
for k in app_name_dict:
    score = fuzz.partial_ratio(k, q)
    scores.append((k, score)) 

In [69]:
scores = sorted(scores, key=lambda x: x[1], reverse=True)

In [70]:
for k, score in scores[:20]:
    filtered_dict[k] = app_name_dict[k]

In [71]:
filtered_dict

defaultdict(list,
            {'TV consoles': ['DuoPower',
              'Universal plug UX',
              'Universal plug UX Green',
              'Expansion plug SX',
              'Expansion plug SX Green',
              'Frame fixing DuoXpand with countersunk head screw',
              'Frame fixing DuoXpand with hexagon head screw',
              'Frame fixing SXRL without safety screw',
              'Frame fixing SXRL with countersunk head screw',
              'Frame fixing SXRL with hexagon head screw',
              'Frame fixing SXR without safety screw',
              'Frame fixing SXR with countersunk head',
              'Frame fixing SXR with hexagon head'],
             'etc.': ['PowerFast FPF II CZP BC', 'PowerFast FPF II CZF BC'],
             'Soft woods': ['Wood drill bit D-WS',
              'Auger bit D-WL',
              'Forstner drill bit D-WFo',
              'Multi-hole saw HS-Multi'],
             'as well as edge connections': ['Premium all-weather sealant

In [72]:
c = convert_dict2context(filtered_dict)

In [73]:
c

'Application of products: \nTV consoles:DuoPower,Universal plug UX,Universal plug UX Green,Expansion plug SX\netc.:PowerFast FPF II CZP BC,PowerFast FPF II CZF BC\nSoft woods:Wood drill bit D-WS,Auger bit D-WL,Forstner drill bit D-WFo,Multi-hole saw HS-Multi\nas well as edge connections:Premium all-weather sealant DDK,Roof sealing compound DD\nGates and doors:Frame fixing DuoXpand with countersunk head screw,Frame fixing DuoXpand with hexagon head screw,Frame fixing SXRL without safety screw,Frame fixing SXRL with countersunk head screw\nWall cabinets:DuoPower,Frame fixing SXR with hook\nand FIS Green:Threaded rod RG M,Threaded rod FIS A,Injections anchor sleeve FIS H K\nNatural stone:Hammer drill bit Quattric II,Hammer drill bit SDS Plus II Pointer,Hammer drill bit SDS Max II / SDS Max IV,Hollow drill bit FHD\nMineral / glass wool:Insulation support DHK 45,Insulation support DHK,Insulation support DHM\nSolid brick:Hammer drill bit Quattric II,Hammer drill bit SDS Plus II Pointer,Hamme

In [49]:
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [74]:
len(tokenizer(c)['input_ids'])

778